# Ejercicio: Ranking Elo aplicado a resultados de fútbol
**Autor**: José A. Troyano.      **Revisor**: Fermín Cruz, Beatriz Pontes, Mariano González, Toñi Reina.     **Última modificación:** 26/11/2018

El sistema de puntuación Elo es un método estadístico para calcular una clasificación entre competidores que no necesariamente se han enfrentado todos entre sí. Empezó a usarse en ajedrez, aunque su uso se está generalizando a otras competiciones. Fue desarrollado por el físico estadounidense de origen húngaro Árpád Élő (1903-1992), y a él debe su nombre. 

Tradicionalmente el ranking Elo ha sido sinónimo de la clasificación mundial de ajedrez (<a href="https://es.wikipedia.org/wiki/Ranking_FIDE">ranking FIDE</a>) y más recientemente otras federaciones deportivas han adoptado un sistema basado en Elo para elaborar sus rankings. Por ejemplo, la FIFA usa Elo para elaborar el ranking de selecciones de <a href="https://es.wikipedia.org/wiki/Clasificaci%C3%B3n_mundial_de_la_FIFA">fútbol femenino</a>, aunque para el masculino sigue usando un método más convencional basado principalmente en el número de partidos ganados recientemente.

En este ejercicio vamos a utilizar los resultados de partidos de fútbol de primera y segunda división españolas para calcular a partir de ellos un ranking de equipos basado en el sistema Elo. Los datos utilizados han sido recuperados de la web de resultados históricos de <a href="http://www.laliga.es/estadisticas-historicas">LaLiga</a> y han sido adaptados para representarlos en un formato <a href="https://es.wikipedia.org/wiki/Valores_separados_por_comas">CSV</a> que resulta muy fácil de manejar desde Python. 

## 1. Carga de datos

Se dispone de los resultados de primera y segunda división desde la temporada 00-01 hasta la temporada 15-16. Los datos están organizados en dos carpetas (<code>Primera</code> y <code>Segunda</code>), y dentro de cada una de ellas hay un fichero <code>CSV</code> para cada temporada. Cada línea de estos ficheros se corresponde con un partido, y en ella se incluyen cinco datos:

- Fecha del partido
- Equipo que juega en casa
- Goles del equipo de casa
- Equipo visitante
- Goles del equipo visitante

Estan serían, por ejemplo, las primeras líneas del fichero de la temporada 15-16 en primera división:

<pre>
        21/08/2015,Málaga CF,0,Sevilla FC,0
        21/08/2015,Atlético de Madrid,1,UD Las Palmas,0
        21/08/2015,RC Deportivo,0,Real Sociedad,0
        21/08/2015,RCD Espanyol,1,Getafe CF,0
        21/08/2015,Rayo Vallecano,0,Valencia CF,0
        21/08/2015,Athletic Club,0,FC Barcelona,1
        21/08/2015,Real Betis,1,Villarreal CF,1
</pre>

Los principales aspectos que tendremos que resolver a la hora de procesar estos datos de entrada serán: separar adecuadamente los campos mediante las comas, interpretar el formato de las fechas para extraer el día, el mes y el año, y leer todos los ficheros de una carpeta para obtener una lista con todos los partidos de una competición.

Para resolver estos problemas haremos uso de algunas utilidades disponibles en la librería estándar de Python. En concreto, antes de empezar, deberemos importar los siguientes elementos:

In [ ]:
from os import listdir
import csv
from collections import namedtuple
from datetime import datetime

### 1.1. Funciones de lectura de datos

La siguiente funnción será la encargada de leer un fichero correspondiente a una temporada, y construir a partir de él una estructura de datos en memoria. Nos apoyaremos en un tipo <code>namedtuple</code> llamado <code>Partido</code> para estructurar la información de cada partido.

In [ ]:
Partido = namedtuple('Partido', 'fecha local goles_local visitante goles_visitante')

def lee_temporada(fichero):
    ''' Lee el fichero de una temporada y devuelve una lista de partidos
    
    ENTRADA: 
       - fichero: nombre del fichero del que se quieren leer los datos -> str
    SALIDA: 
       - lista de partidos -> [Partido(datetime.date, str, int, str, int)] 

    Cada partido se representa con una tupla con los siguientes valores:
    - fecha
    - equipo local
    - goles equipo local
    - equipo visitante
    - goles equipo visitante
    Usaremos el módulo csv de la librería estándar de Python para leer los
    ficheros de entrada.
    En el caso de las fechas, nos hará falta convertir una cadena de caracteres
    en una fecha (objeto date del módulo datetime). Para ello utilizaremoos el 
    método strptime del módulo datetime de la librería estándar de Python.
    En concreto, para crear un objeto date a partir de una cadena_fecha con el
    formato que tenemos en nuestros ficheros de datos, podemos usar la siguiente
    instrucción:
    
        datetime.strptime(fecha, "%d/%m/%Y").date()
        
    También hay que realizar un pequeño procesamiento con los goles. Hay que
    pasar del formato cadena (que es lo que se interpreta al leer el csv) a un
    valor numérico (para poder aplicar operaciones matemáticas si fuese 
    necesario).
    '''
    pass

In [ ]:
# Test de la función lee_temporada
primera_15_16 = lee_temporada('./data/Primera/15-16.csv')
print(primera_15_16[:5])

Apoyándonos en la función <code>lee_temporada</code> implementaremos la función <code>lee_competicion</code> que lee todos los ficheros <code>CSV</code> que encuentre en la carpeta que recibe como parámetro:

In [ ]:
def lee_competicion(carpeta):
    ''' Lee todas las temporadas y devuelve una lista de partidos
    
    ENTRADA: 
       - carpeta: ruta de la carpeta en la que se encuentran los ficheros -> str
    SALIDA: 
       - lista de partidos -> [Partido(datetime.date, str, int, str, int)] 
    
    Toma como entrada la ruta de una carpeta en la que hay varios
    ficheros correspondientes a distintas temporadas de una misma
    competición.
    Devuelve como salida la lista de partidos resultante tras unir
    las listas que construye la función lee_temporada() a partir
    de cada fichero de la carpeta.
    '''
    pass

In [ ]:
# Test de la función lee_competición
primera = lee_competicion('./data/Primera/')
segunda = lee_competicion('./data/Segunda/')
partidos = primera + segunda
print(len(partidos))
print(len(primera), primera[:5])
print(len(segunda), segunda[:5])

### 1.2. Funciones de consulta

En esta sección veremos una serie de funciones que nos permitirán filtrar y extraer informaciones de la estructura de datos que estamos manejando para representar los partidos (una lista de tuplas). Utilizaremos estas funciones de _consulta_ en distintos puntos del resto del ejercicio.

La primera función de este tipo es <code>equipos_participantes</code>. Producirá un conjunto con los nombres de los equipos que han aparecido al menos una vez en una lista de partidos:

In [ ]:
def equipos_participantes(partidos):
    ''' Equipos participantes en una lista de partidos
    
    ENTRADA: 
       - partidos: lista de partidos -> [Partido(datetime.date, str, int, str, int)]
    SALIDA: 
       - conjunto de equipos -> {str} 
    
    Calcula una lista con los nombres de los equipos que han aparecido
    al menos una vez en la lista de partidos que recibe como entrada. 
    '''
    pass

In [ ]:
# Test de la función equipos_participantes
equipos_primera = equipos_participantes(primera)
print(len(equipos_primera), equipos_primera)
equipos_segunda = equipos_participantes(segunda)
print(len(equipos_segunda), equipos_segunda)

Las otras dos funciones de apoyo van a ser de _filtrado_. Con ellas seleccionaremos un subconjunto de la colección de partidos en base a ciertas condiciones, como la fecha en la que se jugaron o los equipos que participaron:

In [ ]:
def partidos_por_fecha(partidos, inicio=None, fin=None):
    ''' Filtra los partidos jugados en un rango de fechas
    
    ENTRADA: 
       - partidos: lista de partidos -> [Partido(datetime.date, str, int, str, int)]
       - inicio: fecha inicial del rango -> datetime.date
       - fin: fecha final del rango -> datetime.date
    SALIDA: 
       - lista de partidos seleccionados -> [Partido(datetime.date, str, int, str, int)] 
    
    Se devuelven aquellos partidos que se han jugado entre las fechas inicio
    y fin. Ambos parámetros serán objetos date del módulo datetime.
    Si inicio es None, se incluirán los partidos desde el principio de
    la serie, y si fin es None se inlcuirán los partidos hasta el último de
    la serie.
    '''
    pass

In [ ]:
# Test de la función partidos_por_fecha
inicio = datetime(2007, 9, 15).date()
fin = datetime(2008, 7, 1).date()
print(len(partidos_por_fecha(partidos, inicio, fin)))
print(len(partidos_por_fecha(partidos, inicio, None)))
print(len(partidos_por_fecha(partidos, None, fin)))
print(len(partidos_por_fecha(partidos, None, None)))

In [ ]:
def partidos_por_equipos(partidos, equipos):
    ''' Filtra los partidos jugados por un conjunto de equipos
    
    ENTRADA: 
       - partidos: lista de partidos -> [Partido(datetime.date, str, int, str, int)]
       - equipos: equipos de los que se requieren los partidos -> [str]
    SALIDA: 
       - lista de partidos seleccionados -> [Partido(datetime.date, str, int, str, int)] 
    
    Se devuelven aquellos partidos que se han jugado por alguno de los
    equipos incluidos en la lista que se recibe como segundo parámetro.
    '''
    pass

In [ ]:
# Test de la función partidos_por_equipos
de_madrid = ['Real Madrid', 'Rayo Vallecano', 'Atlético de Madrid', 'Getafe CF', 'CD Leganés']
partidos_de_madrid = partidos_por_equipos(partidos, de_madrid)
print(len(partidos_de_madrid), partidos_de_madrid[:5])

## 2. Cálculo del ranking Elo

La puntuación Elo de un jugador se calcula a partir de los resultados de sus enfrentamientos con otros jugadores. La idea básica es que la diferencia de puntuación de los contrincantes da, a priori, distintas probabilidades de victoria a cada uno de ellos. Es lo que se denomina puntuación esperada. Por ejemplo, dadas las puntuaciones Elo $R_A$ y $R_B$ de dos jugadores $A$ y $B$, las puntuaciones esperadas $E_A$ y $E_B$ ante un enfrentamiento entre ellos, se calculan con la siguiente fórmula:

$$
E_A = \frac{1}{1+10^{(R_B-R_A)/400}}   \qquad\qquad  E_B = \frac{1}{1+10^{(R_A-R_B)/400}}
$$

$E_A$ y $E_B$ son probabilidades y, por tanto, valores entre $0$ y $1$. Además, se cumple que  $E_A+E_B=1$. 

La puntuación Elo se incrementa, o disminuye, tras un enfrentamiento según si el resultado es mejor o peor de lo que era previsible en función de las puntuaciones esperadas de los contrincantes. Se aplica un ajuste lineal simple, proporcional a la diferencia entre la puntuación esperada y la obtenida por un contrincante. Para poder comparar las puntuaciones esperadas $E_A$ y $E_B$, y los resultados reales $S_A$ y $S_B$, estos últimos se codifican así:
- $1$ en caso de victoria
- $0$ en caso de derrota
- $0.5$ en caso de empate

Dadas las puntuaciones esperadas $E_A$ y $E_B$, y los resultados $S_A$ y $S_B$, los nuevos valores de puntuación Elo para ambos contrincantes se calculan de la siguiente forma:

$$
R'_A = R_A + (S_A - E_A) * k  \qquad\qquad  R'_B = R_B + (S_B - E_B) * k
$$

Donde $k$ es un parámetro que regula la estabilidad del ranking y su velocidad de convergencia. En ajedrez, por ejemplo, se usa $k=16$ para maestros y $k=32$ para jugadores de nivel menor, dando así mayor velocidad a la competición entre aficionados que al ranking de los maestros.

### 2.1. Cáculo de puntuación tras un enfrentamiento

Esta es la función central del cálculo de la puntuación Elo. Su implementación es bastante simple, solo tendremos que _traducir_ la explicación anterior desde un lenguaje matemático a Python:

In [ ]:
def calcula_elo(elo_a, elo_b, goles_a, goles_b, k=20):
    ''' Cálculo de los nuevos valores elo tras un partido
    
    ENTRADA: 
       - elo_a: puntos Elo del equipo A antes del partido -> float
       - elo_b: puntos Elo del equipo B antes del partido -> float
       - goles_a: goles del equipo A en el partido -> int
       - goles_b: goles del equipo B en el partido -> int
       - k: parámetro para regular la velocidad de cambio en el ranking -> float
    SALIDA: 
       - nueva puntuación Elo del equipo A -> float
       - nueva puntuación Elo del equipo B -> float
    
    Dados dos participantes A y B, con puntuación ELOA y ELOB, respectivamente,
    las probabilidades de que cada uno de ellos gane el enfrentamiento se
    calculan así:
        EA = 1/(1+pow(10,(ELOB-ELOA)/400))
        EB = 1/(1+pow(10,(ELOA-ELOB)/400))
    A partir de EA y EB, se calculan los nuevos valores de ELOA y ELOB de la
    siguiente forma:
        ELOA' = ELOA + (RA - EA) * k
        ELOB' = ELOB + (RB - EB) * k
    Donde k es un parámetro que regula la estabilidad del ranking y su
    velocidad de convergencia (usaremos k=20 por defecto) y RA y RB es el 
    resultado de cada partido (1=victoria, 0.5=empate, 0=derrota).
    '''
    pass

In [ ]:
# Test de la función calcula_elo
print(calcula_elo(1000, 1000, 3, 3)) # Equipos parejos, empatan
print(calcula_elo(1000, 1000, 3, 0)) # Equipos parejos, gana uno
print(calcula_elo(2000, 1000, 3, 0)) # Equipos no parejos, gana el mejor 
print(calcula_elo(2000, 1000, 3, 3)) # Equipos no parejos, empatan 
print(calcula_elo(2000, 1000, 0, 3)) # Equipos no parejos, gana el peor 

### 2.3. Cálculo y listado de rankings

En esta sección implementaremos dos funciones que nos permitirán calcular un ranking Elo a partir de una secuencia de partidos, y también mostrar dicho ranking ordenado.

Desde el punto de vista del diseño de datos, este es el momento en el que tendremos que decidir cómo modelar (organizar información) el concepto de ranking. Lo haremos mediante un diccionario, que nos permitirá asociar a cada competidor su puntuación Elo en cada momento.

In [ ]:
def muestra_ranking(ranking, limite=None):
    ''' Muestra un ranking ordenado de mayor a menor
    
    ENTRADA: 
       - ranking: puntuación Elo para cada equipo -> {str: float}
       - limite: número máximo de equipos a mostrar
    SALIDA EN PANTALLA:
       - listado ordenado de los equipos y su puntuación Elo
       
    Toma como entrada un ranking (diccionario {equipo,puntos}) y calcula
    a partir de él una lista de tuplas (puntos, equipo) ordenada de mayor
    a menor por puntuación.
    El parámetro limite establece el numero de elementos del ranking ordenado
    que se mostrará como salida. Si limite es None, se mostrará el ranking
    completo.
    El listado de salida debe incluir para cada equipo su posición en el
    ranking, su nombre y su puntuación Elo. Se puede usar el método format
    de las cadenas de caracteres de Python para dar un buen formato a este
    listado. Hay una buena explicación sobre la notación usada por el
    método format en:
        https://pyformat.info/
    '''
    pass

In [ ]:
# Test de la función muestra_ranking
ranking =  {'CD San Roque': 1489, 'Real Balompédica Linense': 1912, 
            'UD Los Barrios': 1636, 'Algeciras CF': 1750}
muestra_ranking(ranking)

In [ ]:
def calcula_ranking_elo(partidos, ranking_previo=dict()):
    ''' Ranking Elo de todos los equipos tras una secuencia de partidos
    
    ENTRADA: 
       - partidos: lista de partidos -> [Partido(datetime.date, str, int, str, int)]
       - ranking_previo: puntuación Elo inicial para los equipos -> {str: float}
    SALIDA: 
       - ranking actualizado tras los partidos -> {str: float} 
    
    El resultado del ranking será un diccionario en el que las claves serán
    los equipos y los valores serán las puntuaciones de los equipos.
    
    De inicio se asigna a todos los equipos la misma puntuación Elo. Este
    valor inicial puede ser cualquiera. En ajedrez, por ejemplo, se suele
    utilizar el valor 1500 para aquellos jugadores de los que no se tiene
    aún ninguna información. Nostros usaremos el valor 1000. Además, cabe
    la posibilidad de recibir como parámetro un ranking previo. En ese caso
    las entradas de ese diccionario prevalecerán sobre el valor inicial
    por defecto. 
    
    Los partidos deben procesarse en orden cronológico. Para ello, antes
    de empezar, deberá ordenarse la lista de partidos por este criterio.
    '''
    pass

In [ ]:
# Test de la función calcula_ranking_elo
elo = calcula_ranking_elo(primera)
muestra_ranking(elo, limite=10)
print()
elo = calcula_ranking_elo(primera, ranking_previo={'Cádiz CF':2000})
muestra_ranking(elo, limite=10)

## 3. Cálculo del rendimiento en una competición

El rendimiento es una puntuación hipotética que se calcula a partir de los resultados obtenidos en una competición. Por ejemplo, sirve para determinar como se ha comportado un jugador durante un torneo, independientemente del efecto que haya supuesto en su puntuación en el ranking Elo global.

Hay varias formas de calcular el rendimiento a partir de un conjunto de enfrentamientos. Una de las más usadas es la que se conoce como _la regla del 400_. Según esta regla, el rendimiento de un jugador se calcula de la siguiente forma:
- Sumar la puntuación Elo de todos los contrincantes con los que se ha enfrentado
- Restar 400 por cada partido perdido
- Sumar 400 por cada partido ganado
- Dividir por el número total de partidos

In [ ]:
def calcula_rendimiento(equipo, partidos, ranking_elo):
    ''' Cálculo del rendimiento de un equipo en un conjunto de partidos
    
    ENTRADA:
       - equipo: equipo del que se quiere calcular el rendimiento -> str
       - partidos: lista de partidos de la competición -> [Partido(datetime.date, str, int, str, int)]
       - ranking_elo: puntuación Elo para los equipos -> {str: float}
    SALIDA: 
       - rendimiento calculado para el equipo -> float 
    
    Toma como entrada un equipo, una lista de partidos y un ranking Elo. 
    Calcula como salida el rendimiento del equipo en función de los partidos
    de la lista en los que ha participado. Se usa para ello la siguiente
    fórmula:
    
               suma Elo de competidores + victorias*400 - derrotas*400
       rend = ---------------------------------------------------------
                             número de partidos
    
    Una buena forma de enfocar la implementación es calcular, a partir de la
    lista de partidos, una lista tuplas (contrincante, diferencia de goles).
    Solo con el signo de la diferencia se puede determinar si para ese
    contrincante hay que añadir 400, -400 ó 0 al acumular su puntuación Elo.
    Según el enfoque, puede venir bien usar una función que calcule el signo
    de un número entero. Pero en Python no está disponible, ¿cuál es el 'idiom'
    Python para calcular el signo de un número? 
    '''
    pass

In [ ]:
# Test de la función calcula_rendimiento
inicio_14 = datetime(2014, 8, 15).date()
fin_16 = datetime(2016, 7, 1).date()
partidos_hasta_14 = partidos_por_fecha(partidos, inicio=None, fin=inicio_14)
ranking_hasta_14 = calcula_ranking_elo(partidos_hasta_14)
primera_14_16 = partidos_por_fecha(partidos, inicio=inicio_14, fin=None)
print(calcula_rendimiento('Real Madrid', primera_14_16, ranking_hasta_14))

Como último ejercicio de este notebook, nos apoyaremos en la función <code>calcula_rendimiento</code> para calcular un ranking de rendimiento. Para ello, implementaremos la función <code>calcula_rankig_rendimiento</code> que toma como entrada una lista de equipos, una lista de partidos sobre la que se calculará el rendimiento de los equipos y un ranking Elo que proporcionará la base de puntuación necesaria para calcular el rendimiento de cada equipo.

In [ ]:
def calcula_ranking_rendimiento(equipos, partidos, ranking_elo):
    ''' Cálculo del ranking de rendimiento de un conjunto de equipos
    
    ENTRADA:
       - equipos: equipos de los que se quiere calcular el rendimiento -> [str]
       - partidos: lista de partidos de la competición -> [Partido(datetime.date, str, int, str, int)]
       - ranking_elo: puntuación Elo inicial para los equipos -> {str: float}
    SALIDA: 
       - ranking actualizado tras los partidos -> {str: float} 
    
    Toma como entrada una lista de equipos, una lista de partidos en
    los que pueden haber participado esos equipos y un ranking_elo.
    Produce como salida un ranking en el que a cada equipo de la
    lista equipos se le asocia su rendimiento en función de los
    partidos en los que ha participado.
    '''
    pass

In [ ]:
# Test de la función calcula_ranking_rendimiento
andaluces = ['Málaga CF', 'Real Betis', 'Sevilla FC', 'RC Recreativo', 
            'Cádiz CF', 'Xerez CD', 'Granada CF', 'Córdoba CF']
rendimiento = calcula_ranking_rendimiento(andaluces, primera_14_16, ranking_hasta_14)
muestra_ranking(rendimiento)